# Exploratory data analysis HUPA-UCM Diabetes 
<br>  Morales Martinez Arturo 
<br>  Luis Daniel Moreno Santamaria

## Preprocessed Data 

### Number of rows by patient

In [1]:
import os 
import pandas as pd
import matplotlib.pyplot as plt
import re
from tabulate import tabulate

Just to find patient's number

In [11]:
def find_number_string(string):
    numbers = re.findall(r'\d+', string)
    return numbers

The following cell is intended to get a dictionary where each key is the number of patient 

In [12]:
data_dir = 'HUPA-UCM Diabetes Dataset/Preprocessed'
patients = {int(find_number_string(os.path.join(data_dir, filename))[0]) : pd.read_csv(os.path.join(data_dir, filename), sep=';').shape[0]
                   for filename in os.listdir(data_dir)}

### Just to sort by the amount of rows
patients = dict(sorted(patients.items(), key=lambda item: item[1]))

In [21]:
rows = [(key, value) for key, value in patients.items()]

Now, we have the number of rows by each patient

In [29]:
table = []
sum_days = 0 
for i, row in enumerate(rows):
    patient_id = row[0]
    rowss = row[1]
    days = row[1] / 288
    if i!=24 and i!=23 and i!=22:
        sum_days += days
        print(i, rowss)
    months = days // 30
    years = round(days / 30 / 12, 2)
    #table.append([patient_id, rowss, round(days, 1), int(months), years])

print(f'Days average: {sum_days/len(rows)}')
#print(tabulate(table, headers=["Patient", 'Rows', "Days", "Months", "Years"]))


0 2290
1 2343
2 2862
3 2902
4 2976
5 3181
6 3184
7 3599
8 3711
9 3770
10 3792
11 3812
12 3829
13 3835
14 3839
15 3857
16 3858
17 3895
18 3919
19 4006
20 4023
21 4096
Days average: 10.774861111111111


In [7]:
plt.figure(figsize=(10, 10))
plt.barh([f'Patient {r[0]}' for r in rows], [r[1] for r in rows])
plt.xlabel('Number of rows')
plt.ylabel('Patients')
plt.grid(True)
plt.title('Number of rows for each patient')
plt.savefig('Images/Number_rows_by_patient')

TypeError: 'int' object is not iterable

<Figure size 1000x1000 with 0 Axes>